In [1]:
from tqdm import tqdm
import pandas as pd
import datasets
from pathlib import Path
from transformers import Pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from typing import List, Optional, Dict
import pickle
import torch
from torch.utils.data import DataLoader
from functools import lru_cache
from nltk.stem.porter import PorterStemmer
from collections import defaultdict

tqdm.pandas()

%load_ext autoreload
%autoreload 2

from pywikidata import Entity
from kbqa.utils.train_eval import get_best_checkpoint_path
from kbqa.caches.ner_to_sentence_insertion import NerToSentenceInsertion

from trie import MarisaTrie

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-01-08 19:06:59.867700: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-08 19:07:00.038280: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-08 19:07:00.070643: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register 

In [2]:
!git clone https://github.com/askplatypus/wikidata-simplequestions.git

test_df = pd.read_csv(
    "./wikidata-simplequestions/annotated_wd_data_test_answerable.txt",
    sep="\t",
    names=["S", "P", "O", "Q"],
)
a
train_df = pd.read_csv(
    "./wikidata-simplequestions/annotated_wd_data_train_answerable.txt",
    sep="\t",
    names=["S", "P", "O", "Q"],
)

valid_df = pd.read_csv(
    "./wikidata-simplequestions/annotated_wd_data_valid_answerable.txt",
    sep="\t",
    names=["S", "P", "O", "Q"],
)

fatal: destination path 'wikidata-simplequestions' already exists and is not an empty directory.


In [3]:
# import numpy as np

# filtered_test = np.load('simple_questions_filtered.npy')
# test_df = pd.DataFrame(filtered_test, columns=["S", "P", "O", "Q"])
# test_df.head()

In [4]:
class MGENREPipeline(Pipeline):
    """MGENREPipeline - HF Pipeline for mGENRE EntityLinking model"""

    def _sanitize_parameters(self, **kwargs):
        forward_kwargs = {}
        if "num_beams" in kwargs:
            forward_kwargs["num_beams"] = kwargs.get("num_beams", 20)
        if "num_return_sequences" in kwargs:
            forward_kwargs["num_return_sequences"] = kwargs.get(
                "num_return_sequences", 20
            )
        if "mgenre_trie" in kwargs:
            forward_kwargs["mgenre_trie"] = kwargs.get("mgenre_trie")
        return {}, forward_kwargs, {}

    def preprocess(self, input_):
        return self.tokenizer(
            input_,
            return_tensors="pt",
        )

    def _forward(
        self,
        input_tensors,
        num_beams=10,
        num_return_sequences=10,
        mgenre_trie=None,
    ):
        outputs = self.model.generate(
            **{k: v.to(self.device) for k, v in input_tensors.items()},
            num_beams=num_beams,
            num_return_sequences=num_return_sequences,
            prefix_allowed_tokens_fn=lambda batch_id, sent: mgenre_trie.get(
                sent.tolist()
            )
            if mgenre_trie is not None
            else None,
        )
        return outputs

    def postprocess(self, model_outputs):
        outputs = self.tokenizer.batch_decode(
            model_outputs,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False,
        )
        return outputs


class EntitiesSelection:
    def __init__(self, ner_model):
        self.stemmer = PorterStemmer()
        self.ner_model = ner_model

    def entities_selection(self, entities_list, mgenre_predicted_entities_list):
        final_preds = []

        for pred_text in mgenre_predicted_entities_list:
            labels = []
            try:
                _label, lang = pred_text.split(" >> ")
                if lang == "en":
                    labels.append(_label)
            except Exception as e:
                print(f"Error {str(e)} with pred_text={pred_text}")

            if len(labels) > 0:
                for label in labels:
                    label = label.lower()
                    if self._check_label_fn(label, entities_list):
                        final_preds.append(pred_text)

        return list(dict.fromkeys(final_preds))

    @lru_cache(maxsize=8192)
    def _label_format_fn(self, label):
        return " ".join(
            [self.stemmer.stem(str(token)) for token in self.ner_model(label)]
        )

    def _check_label_fn(self, label, entities_list):
        label = self._label_format_fn(label)
        for entity in entities_list:
            entity = self._label_format_fn(entity)
            if label == entity:
                return True
        return False

In [5]:
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

mgenre_tokenizer = AutoTokenizer.from_pretrained("facebook/mgenre-wiki")
mgenre_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mgenre-wiki").eval()

# with open('./wdsq_mgenre_trie.pkl', 'rb') as f:
#     trie = pickle.load(f)

mgenre = MGENREPipeline(
    mgenre_model,
    mgenre_tokenizer,
    # mgenre_trie=trie,
    device=device,
)


ner = NerToSentenceInsertion(model_path="../../../ner/")
entities_selection_module = EntitiesSelection(ner.model)

RuntimeError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 5; 10.76 GiB total capacity; 1.15 GiB already allocated; 16.81 MiB free; 1.17 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [7]:
df = train_df

final_results = defaultdict(list)

for q in tqdm(df["Q"].values):
    text_with_labeling, entities_list = ner.entity_labeling(q, True)
    mgenre_results = mgenre(text_with_labeling)
    selected_mgenre_results = entities_selection_module.entities_selection(
        entities_list, mgenre_results
    )

    entities = []
    for lbl in selected_mgenre_results:
        try:
            lbl = lbl.split(" >> ")[0]
        except:
            lbl = lbl[: lbl.index(" >>")]
            lbl = lbl[: lbl.index(">>")]
        try:
            entities.extend(Entity.from_label(lbl))
        except ValueError:
            pass

    final_results["Q"].append(q)
    final_results["mgenre_results"].append(mgenre_results)
    final_results["selected_mgenre_results"].append(selected_mgenre_results)
    final_results["selected_entities"].append(entities)


final_df = pd.concat([df, pd.DataFrame(final_results)], axis=1)
final_df["selected_entities"] = final_df["selected_entities"].apply(
    lambda x: [_x.idx for _x in x]
)

final_df.to_pickle("wdsq_train_with_mgenre_no_prefix_tree.pkl")
final_df.to_csv("wdsq_train_with_mgenre_no_prefix_tree.csv", index=False)

  0%|          | 0/19481 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
  0%|          | 4/19481 [00:01<2:07:33,  2.54it/s]

Error not enough values to unpack (expected 2, got 1) with pred_text=List of American football players who have taken five-wicket hauls on Test match >>
Error not enough values to unpack (expected 2, got 1) with pred_text=List of American football players who have taken five-wicket hauls on their careers >>
Error not enough values to unpack (expected 2, got 1) with pred_text=List of American football players who have taken five-wicket hauls on Test debut >>
Error not enough values to unpack (expected 2, got 1) with pred_text=List of American football players who have taken five-wicket hauls on Test match squ


  0%|          | 10/19481 [00:06<3:06:56,  1.74it/s]/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1036: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
  0%|          | 35/19481 [00:20<2:57:01,  1.83it/s]

Error not enough values to unpack (expected 2, got 1) with pred_text=Communist Party of Britain (Marxist–Leninist) (historical)
Error not enough values to unpack (expected 2, got 1) with pred_text=Communist Party of Britain (Marxist–Leninist) Commonwealth Federation >>


  0%|          | 61/19481 [00:34<2:33:14,  2.11it/s]

...

Error not enough values to unpack (expected 2, got 1) with pred_text=List of settlements in the London Borough of Hammersmith and Fulham >>


  6%|▌         | 1078/19481 [10:13<2:36:12,  1.96it/s]

Error not enough values to unpack (expected 2, got 1) with pred_text=ローソン・ロビンソン (初代ロビンソン男爵)
Error not enough values to unpack (expected 2, got 1) with pred_text=ローソン・ロビンソン (初代ロビンソン子爵)


  6%|▌         | 1097/19481 [10:27<2:55:20,  1.75it/s]


KeyboardInterrupt: 

In [13]:
final_df = pd.concat([test_df, pd.DataFrame(final_results)], axis=1)
final_df["selected_entities"] = final_df["selected_entities"].apply(
    lambda x: [_x.idx for _x in x]
)

final_df.to_pickle("filtered_test_with_mgenre_prefix_tree.pkl")
final_df.to_csv("filtered_test_with_mgenre_prefix_tree.csv", index=False)

In [14]:
!git clone https://github.com/vladislavneon/RuBQ.git

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
fatal: destination path 'RuBQ' already exists and is not an empty directory.


In [15]:
import json

with open("./RuBQ/RuBQ_2.0/RuBQ_2.0_test.json", "r") as f:
    rubq = json.load(f)

In [16]:
final_results = defaultdict(list)
for rec in tqdm(rubq):
    q = rec["question_eng"]
    text_with_labeling, entities_list = ner.entity_labeling(q, True)
    mgenre_results = mgenre(text_with_labeling)
    selected_mgenre_results = entities_selection_module.entities_selection(
        entities_list, mgenre_results
    )
    entities = []
    for lbl in selected_mgenre_results:
        try:
            lbl = lbl.split(" >> ")[0]
        except:
            lbl = lbl[: lbl.index(" >>")]
            lbl = lbl[: lbl.index(">>")]
        try:
            entities.extend(Entity.from_label(lbl))
        except ValueError:
            pass

    final_results["Q"].append(q)
    final_results["mgenre_results"].append(mgenre_results)
    final_results["selected_mgenre_results"].append(selected_mgenre_results)
    final_results["selected_entities"].append(entities)

  0%|          | 0/2330 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1036: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
  0%|          | 11/2330 [00:03<12:49,  3.01it/s]

Error not enough values to unpack (expected 2, got 1) with pred_text=What Modern State Occupies The Territory Where The Ancient Assyrian City Of
Error not enough values to unpack (expected 2, got 1) with pred_text=What Modern State Occupies The Territory Where The Ancient Assyrian City of
Error not enough values to unpack (expected 2, got 1) with pred_text=What Modern State Occupies the Territory Where the Ancient Assyrian City of
Error not enough values to unpack (expected 2, got 1) with pred_text=What Modern State Occupies, The Territory Where The Ancient Assyrian City
Error not enough values to unpack (expected 2, got 1) with pred_text=What Modern State Occupies the Territory Where The Ancient Assyrian City Of
Error not enough values to unpack (expected 2, got 1) with pred_text=What Modern State Occupies the Territory Where The Ancient Assyrian City of
Error not enough values to unpack (expected 2, got 1) with pred_text=What Modern State Occupies the Territory of the Ancient Assyria

  1%|          | 14/2330 [00:04<09:47,  3.94it/s]

Error too many values to unpack (expected 2) with pred_text=Golf >> en) >> en


  1%|          | 18/2330 [00:05<10:35,  3.64it/s]

...

 98%|█████████▊| 2274/2330 [10:01<00:18,  2.99it/s]

Error not enough values to unpack (expected 2, got 1) with pred_text=What Is the Architectural Style of the Building of the Guggenheim Museum in Bilbao
Error not enough values to unpack (expected 2, got 1) with pred_text=What Is the Architectural Style of the Building Of The Building Of The Guggenheim
Error not enough values to unpack (expected 2, got 1) with pred_text=What Is the Architectural Style of the Building of The Guggenheim Museum in Bilbao
Error not enough values to unpack (expected 2, got 1) with pred_text=What Is the Architectural Style of the Building of Guggenheim Museum in Bilbao?
Error not enough values to unpack (expected 2, got 1) with pred_text=What Is the Architectural Style Of The Building Of The Building Of The Guggenheim
Error not enough values to unpack (expected 2, got 1) with pred_text=What Is the Architectural Style of the Building of the Guggenheim Museum In Bilbao


 99%|█████████▊| 2298/2330 [10:07<00:08,  3.62it/s]

Error too many values to unpack (expected 2) with pred_text=Cologne Cathedral >> en) >> en


100%|██████████| 2330/2330 [10:16<00:00,  3.78it/s]


In [17]:
df = pd.DataFrame(final_results)
df["selected_entities"] = df["selected_entities"].apply(lambda x: [_x.idx for _x in x])

df.to_pickle("test_rubq2_mgenre.pkl")
df.to_csv("test_rubq2_mgenre.csv", index=False)

In [19]:
df.head()

,Q,mgenre_results,selected_mgenre_results,selected_entities
0,What can cause a tsunami?,"[What Can Cause a Tsunami? >> en, What Can Cau...",[],[]
1,"Who wrote the novel ""uncle Tom's Cabin""?","[Uncle Tom's Cabin >> en, Uncle Tom's Cabin (n...","[Uncle Tom's Cabin >> en, Uncle Tom's cabin >>...","[Q2222, Q820452, Q3549210, Q3549212, Q3549214,..."
2,"Who is the author of the play ""Romeo and Juliet""?","[Romeo and Juliet >> en, Romeo and Juliet (Pro...",[Romeo and Juliet >> en],"[Q83186, Q223857, Q517265, Q763716, Q911487, Q..."
3,What is the name of the capital of Romania?,"[Romania >> en, Kingdom of Romania >> en, Soci...",[Romania >> en],"[Q218, Q2175798, Q3441038, Q7362480, Q18487313..."
4,What instrument did Jimi Hendrix play?,"[Jimi Hendrix >> en, Jimi Hendrix >> fr, Jimi ...",[Jimi Hendrix >> en],"[Q5928, Q6199228, Q28453281, Q50734578]"
